In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Параметры
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
NUM_CLASSES = 1  # Для задачи обнаружения номеров

# Создание модели ResNet50
def create_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base_model.trainable = False  # Замораживаем базовую модель

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(NUM_CLASSES, activation='sigmoid')  # Используем sigmoid для бинарной классификации
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Подготовка данных
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'C:/Users/lucky/Desktop/ПП_3_сем/ML_v3/OCR/train',  # Путь к данным
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # Для бинарной классификации
)

# Обучение модели
model = create_model()
model.fit(train_generator, epochs=10)  # Увеличьте количество эпох по необходимости

# Сохранение модели в формате .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Сохранение .tflite модели
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Модель успешно сохранена в формате .tflite")